# Imports

# Creacion Base de Datos - MySql

In [2]:
import boto3
import pandas as pd
import numpy as np
import configparser
import psycopg2


## Leemos archivo de Configuracion y credenciales

In [15]:
config = configparser.ConfigParser()
config.read("config.cfg")  

['config.cfg']

In [18]:
aws_rds_conn = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                            aws_secret_access_key=config.get('IAM', 'SECRET_KEY'),
                            region_name="us-east-2")

In [19]:
rds_intance_ids = []

aws_response = aws_rds_conn.describe_db_instances()

for response in aws_response['DBInstances']:
    rds_intance_ids.append(response['DBInstanceIdentifier'])
    
print(f"intancias disponibles: {rds_intance_ids}")

intancias disponibles: ['ecomercedb', 'hechosdbproject']


## Creamos instancia de DB - MySql

In [20]:
try:

    response = aws_rds_conn.create_db_instance(
                                                DBInstanceIdentifier=config.get('HECHOS', 'DB_INSTANCE_ID'),
                                                DBName=config.get('HECHOS', 'DB_NAME'),
                                                MasterUsername=config.get('HECHOS', 'DB_USERNAME'),
                                                MasterUserPassword=config.get('HECHOS', 'DB_PASSWORD'),
                                                Port=int(config.get('HECHOS', 'DB_PORT')),
                                                DBInstanceClass='db.t3.micro',
                                                Engine='mysql',
                                                PubliclyAccessible=True,
                                                AllocatedStorage=20,
                                                VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')]
                                                                        
    )
except aws_rds_conn.exceptions.DBInstanceAlreadyExistsFault:
    print("La instancia ya existe")
    
except Exception as e:
    print("Error:", e)

La instancia ya existe


## Instancia

In [21]:
try:
    instance = aws_rds_conn.describe_db_instances(DBInstanceIdentifier = config.get('HECHOS', 'DB_INSTANCE_ID'))

    RDS_HOSTNAME = instance.get('DBInstances')[0].get('Endpoint').get('Address')
    print(RDS_HOSTNAME)
    
except Exception as e:
    print("Error:", e)

hechosdbproject.ctiwauocyj1r.us-east-2.rds.amazonaws.com


## Queries
    

In [22]:
import sql_queries_H
sql_queries_H.DDL_QUERY

'-- base de datos hecha en mysql\n\ncreate table Tiempo (\n\n    idfecha int primary key,\n    fecha_completa DateTime,\n    fecha_dia_semana int,\n    fehca_dia_mes int,\n    dia_nombre varchar(10),\n    semana_fin_bandera varchar(13), -- entre semana, fin de semana\n    semana_ano int,\n    dia_semana_inicio datetime,\n    mes int,\n    mes_nombre varchar(10),\n    cuarto int,\n    mes_fiscal int,\n    cuarto_fiscal int,\n    ano_fiscal int,\n    mes_fin_bandera varchar(13), -- fin de mes, mes\n\n);\n\nCREATE TABLE Usuario (\n\n    idusuario int,\n    idrol int,\n    nombre varchar(100),\n    tipo_documento varchar(20),\n    num_documento varchar(20),\n    direccion varchar(20),\n    telefono varchar(20),\n    clave varbinary(MAX),\n    estado bit,\n    rol_nombre varchar(30)\n\n);\n\nCREATE TABLE Articulo (\n\n    idarticulo int,\n    nombre_categoria varchar(30),\n    codigo varchar(50),\n    nombre varchar(100),\n    precio_venta decimal(11,2),\n    stock INT\n\n);\n\nCREATE TABLE

In [24]:
try:
    db_mysql_conn = psycopg2.connect(
                                database = config.get('HECHOS', 'DB_NAME'),
                                user = config.get('HECHOS', 'DB_USERNAME'),
                                password = config.get('HECHOS', 'DB_PASSWORD'),
                                port=int(config.get('HECHOS', 'DB_PORT')),
                                host = RDS_HOSTNAME
    )

    cursor = db_mysql_conn.cursor()

    cursor.execute(sql_queries_H.DDL_QUERY)

    db_mysql_conn.commit()

except Exception as e:
    print("Error:", e)